## Etapa 1: Leitura dos dados e armazenamento 

Para a execução desta primeira etapa, é imprescindível rodar o código **0_setup.ipynb**

In [0]:
TARGET_PATH = "/Volumes/workspace/raw/ifood_case/"

dbutils.fs.mkdirs(TARGET_PATH)

files = [
    "offers.json",
    "profile.json",
    "transactions.json"
]

for f in files:
    dbutils.fs.cp(
        f"/Workspace/ifood-case/data/raw/{f}",
        f"{TARGET_PATH}{f}"
    )

In [0]:
# Leitura dos arquivos brutos e armazenamento
RAW_PATH = TARGET_PATH
dfs = {}

for file in dbutils.fs.ls(RAW_PATH):
    if file.name.endswith('.json'):
        nome = file.name.replace('.json', '')
        dfs[nome] = (
            spark.read
            .option("multiline", "true")
            .json(file.path)
        )

# Dataframes
bronze_offers = dfs.get('offers')
bronze_profile = dfs.get('profile')
bronze_transactions = dfs.get('transactions')


## Etapa 2: Transformação dos dados

**Profile**

Contém atributos de cerca de 17k clientes:
- age (int): idade do cliente na criação da conta
- registeredon (int): data de criação da conta
- gender (string): gênero do cliente
- id (string): id do cliente
- credit_card_limit (float): limite do cartão registrado

In [0]:
# Visualização do DataFrame
bronze_profile.show()

+---+-----------------+------+--------------------+-------------+
|age|credit_card_limit|gender|                  id|registered_on|
+---+-----------------+------+--------------------+-------------+
|118|             NULL|  NULL|68be06ca386d4c319...|     20170212|
| 55|         112000.0|     F|0610b486422d4921a...|     20170715|
|118|             NULL|  NULL|38fe809add3b4fcf9...|     20180712|
| 75|         100000.0|     F|78afa995795e4d85b...|     20170509|
|118|             NULL|  NULL|a03223e636434f42a...|     20170804|
| 68|          70000.0|     M|e2127556f4f64592b...|     20180426|
|118|             NULL|  NULL|8ec6ce2a7e7949b1b...|     20170925|
|118|             NULL|  NULL|68617ca6246f4fbc8...|     20171002|
| 65|          53000.0|     M|389bc3fa690240e79...|     20180209|
|118|             NULL|  NULL|8974fc5686fe429db...|     20161122|
|118|             NULL|  NULL|c4863c7985cf408fa...|     20170824|
|118|             NULL|  NULL|148adfcaa27d485b8...|     20150919|
| 58|     

In [0]:
# Validação: Tamanho da base conforme as orientações
bronze_profile.count()

17000

In [0]:
# Validação: Tipos de dados das colunas do DataFrame
bronze_profile

DataFrame[age: bigint, credit_card_limit: double, gender: string, id: string, registered_on: string]

In [0]:
# Transformação: Converter tipos de dados
from pyspark.sql.functions import to_date, col

silver_profile = (
    bronze_profile
    .withColumn(
        "registered_on",
        to_date(col("registered_on"), "yyyyMMdd") # Campo string para data
    )
    .withColumn(
        "credit_card_limit",
        col("credit_card_limit").cast("decimal(10,2)") # Campo double para decimal
    )
    .withColumn(
        "age",
        col("age").cast("int") # Campo bigint para int
    )
)

# Visualização dos novos tipos de dados
silver_profile

DataFrame[age: int, credit_card_limit: decimal(10,2), gender: string, id: string, registered_on: date]

In [0]:
# Transformação: Remover duplicatas
silver_profile = silver_profile.dropDuplicates()

In [0]:
# Checagem: Quais são os possíveis preenchimentos do campo Gênero? 
display(
    silver_profile.groupBy("gender")
      .count()
      .orderBy("gender")
)
# F: feminino | M: masculino | O: outro | null: não informado
# Campo já normalizado 

gender,count
null,2175
F,6129
M,8484
O,212


In [0]:
# Checagem: Qual a maior e menor idade da base? Preenchimento é consistente?
from pyspark.sql.functions import min, max

silver_profile.select( 
    min("age").alias("min_age"), 
    max("age").alias("max_age")
).show()
# 118 anos não é uma idade esperada

+-------+-------+
|min_age|max_age|
+-------+-------+
|     18|    118|
+-------+-------+



In [0]:
# Checagem: Quantos clientes têm 100 anos ou mais?
from pyspark.sql.functions import col, count

silver_profile.filter(
    col("age") >= 100
).groupBy("age")\
 .count()\
 .orderBy("age")\
 .show()
# 2.192 clientes têm 100 anos ou mais
# 2.175 clientes têm 118 anos (~13%)

+---+-----+
|age|count|
+---+-----+
|100|   12|
|101|    5|
|118| 2175|
+---+-----+



In [0]:
# Checagem: Qual o limite de crédito dos clientes que têm 118 anos? E o gênero?
silver_profile \
    .filter(col("age") == 118) \
    .select("credit_card_limit", "gender") \
    .distinct() \
    .show()
# credit_card_limit e gender não foram informados

+-----------------+------+
|credit_card_limit|gender|
+-----------------+------+
|             NULL|  NULL|
+-----------------+------+



In [0]:
# Transformação: Remover clientes com 118 anos
gold_profile = silver_profile.filter(col("age") != 118)
# 2.175 clientes com 118 foram retirados

gold_profile.count()
# 14.825 clientes na base final

14825

In [0]:
# Visualização da tabela final
gold_profile.show()

+---+-----------------+------+--------------------+-------------+
|age|credit_card_limit|gender|                  id|registered_on|
+---+-----------------+------+--------------------+-------------+
| 46|         73000.00|     F|021c1940868647efb...|   2015-02-21|
| 79|         42000.00|     M|88bea58132ac47379...|   2016-11-17|
| 20|         40000.00|     M|0969a11464224731b...|   2017-11-20|
| 48|         74000.00|     F|91e68a13d7e5471cb...|   2017-09-07|
| 57|         66000.00|     M|5a5f5e54f76249b38...|   2017-08-01|
| 59|         73000.00|     M|303bff336e8449d6a...|   2017-08-07|
| 37|         58000.00|     F|2b484f47c64741889...|   2017-10-13|
| 61|         98000.00|     M|8b70ca1f5f5a45bda...|   2017-11-21|
| 54|         47000.00|     F|f50ee71f3a3c4d169...|   2017-11-23|
| 73|         46000.00|     M|3a94b435ee0647ceb...|   2018-06-10|
| 53|         75000.00|     F|bb465e90882143b6a...|   2015-08-07|
| 34|         33000.00|     F|b56a93598fa5432b9...|   2017-11-03|
| 47|     

**Offers**

Contém os ids das ofertas e metadados de cada uma delas:
- id (string): id da oferta
- offer_type (string): o tipo da oferta (BOGO, discount, informational)
- min_value (int): valor mínimo para ativação da oferta
- duration (int): duração da oferta
- discount_value (int): valor do desconto
- channels (list of strings): canais de veiculação

In [0]:
# Visualização do DataFrame
bronze_offers.show()

+--------------------+--------------+--------+--------------------+---------+-------------+
|            channels|discount_value|duration|                  id|min_value|   offer_type|
+--------------------+--------------+--------+--------------------+---------+-------------+
|[email, mobile, s...|            10|     7.0|ae264e3637204a6fb...|       10|         bogo|
|[web, email, mobi...|            10|     5.0|4d5c57ea9a6940dd8...|       10|         bogo|
|[web, email, mobile]|             0|     4.0|3f207df678b143eea...|        0|informational|
|[web, email, mobile]|             5|     7.0|9b98b8c7a33c4b65b...|        5|         bogo|
|        [web, email]|             5|    10.0|0b1e1539f2cc45b7b...|       20|     discount|
|[web, email, mobi...|             3|     7.0|2298d6c36e964ae4a...|        7|     discount|
|[web, email, mobi...|             2|    10.0|fafdcd668e3743c1b...|       10|     discount|
|[email, mobile, s...|             0|     3.0|5a8bc65990b245e5a...|        0|inf

In [0]:
# Validação: Tipos de dados das colunas do DataFrame
bronze_offers

DataFrame[channels: array<string>, discount_value: bigint, duration: double, id: string, min_value: bigint, offer_type: string]

In [0]:
# Transformação: Converter tipos de dados
from pyspark.sql.functions import to_date, col

silver_offers = (
    bronze_offers
    .withColumn(
        "discount_value",
        col("discount_value").cast("decimal(10,2)") # Campo bigint para decimal
    )
    .withColumn(
        "min_value",
        col("min_value").cast("decimal(10,2)") # Campo bigint para decimal
    )
    .withColumn(
        "duration",
        col("duration").cast("int") # Campo double para int
    )
)

# Visualização dos novos tipos de dados
silver_offers

DataFrame[channels: array<string>, discount_value: decimal(10,2), duration: int, id: string, min_value: decimal(10,2), offer_type: string]

In [0]:
# Transformação: Remover duplicatas
gold_offers = silver_offers.dropDuplicates()

In [0]:
# Visualização da tabela final
gold_offers.show()

+--------------------+--------------+--------+--------------------+---------+-------------+
|            channels|discount_value|duration|                  id|min_value|   offer_type|
+--------------------+--------------+--------+--------------------+---------+-------------+
|        [web, email]|          5.00|      10|0b1e1539f2cc45b7b...|    20.00|     discount|
|[web, email, mobi...|          2.00|      10|fafdcd668e3743c1b...|    10.00|     discount|
|[web, email, mobi...|          5.00|       5|f19421c1d4aa40978...|     5.00|         bogo|
|[web, email, mobile]|          5.00|       7|9b98b8c7a33c4b65b...|     5.00|         bogo|
|[web, email, mobi...|         10.00|       5|4d5c57ea9a6940dd8...|    10.00|         bogo|
|[web, email, mobile]|          0.00|       4|3f207df678b143eea...|     0.00|informational|
|[email, mobile, s...|         10.00|       7|ae264e3637204a6fb...|    10.00|         bogo|
|[email, mobile, s...|          0.00|       3|5a8bc65990b245e5a...|     0.00|inf

**Transactions**
Contém cerca de 300k eventos:
- event (str): descrição do evento (transação, oferta recebida, etc.)
- account_id (str): id do cliente
- time_since_test_start (int): tempo desde o começo do teste em dias (t=0)
- value (json): registra offer_id, desconto (reward) ou valor da transação

In [0]:
# Visualização do DataFrame
bronze_transactions.show(truncate=False)

+--------------------------------+--------------+---------------------+----------------------------------------------------+
|account_id                      |event         |time_since_test_start|value                                               |
+--------------------------------+--------------+---------------------+----------------------------------------------------+
|78afa995795e4d85b5d9ceeca43f5fef|offer received|0.0                  |{NULL, 9b98b8c7a33c4b65b9aebfe6a799e6d9, NULL, NULL}|
|a03223e636434f42ac4c3df47e8bac43|offer received|0.0                  |{NULL, 0b1e1539f2cc45b7b9fa7c272da2e1d7, NULL, NULL}|
|e2127556f4f64592b11af22de27a7932|offer received|0.0                  |{NULL, 2906b810c7d4411798c6938adc9daaa5, NULL, NULL}|
|8ec6ce2a7e7949b1bf142def7d0e0586|offer received|0.0                  |{NULL, fafdcd668e3743c1bb461111dcafc2a4, NULL, NULL}|
|68617ca6246f4fbc85e91a2a49552598|offer received|0.0                  |{NULL, 4d5c57ea9a6940dd891ad53e9dbe8da0, NULL, NULL}|


In [0]:
# Validação: Tamanho da base conforme as orientações
bronze_transactions.count()

306534

In [0]:
# Validação: Tipos de dados das colunas do DataFrame
bronze_transactions

DataFrame[account_id: string, event: string, time_since_test_start: double, value: struct<amount:double,offer id:string,offer_id:string,reward:double>]

In [0]:
# Análise: Qual a diferença dos campos offer_id e offer id?
from pyspark.sql.functions import col, when, count

bronze_profile_analysis = (
    bronze_transactions
        .select(
            "event",
            col("value.`offer id`").alias("offer_id_1"),
            col("value.offer_id").alias("offer_id_2")
        )
        .groupBy("event")
        .agg(
            count(when(col("offer_id_1").isNotNull(), True)).alias("qt_offer_id_1"),
            count(when(col("offer_id_2").isNotNull(), True)).alias("qt_offer_id_2")
        )
)

bronze_profile_analysis.show()
# Os campos offer_id e offer id não são preenchidos ao mesmo tempo.
    # Ajuste a ser realizado: juntar os campos em uma única coluna

+---------------+-------------+-------------+
|          event|qt_offer_id_1|qt_offer_id_2|
+---------------+-------------+-------------+
| offer received|        76277|            0|
|   offer viewed|        57725|            0|
|offer completed|            0|        33579|
|    transaction|            0|            0|
+---------------+-------------+-------------+



In [0]:
# Transformação: Extração dos valores do struct
from pyspark.sql.functions import col, coalesce

silver_transactions = bronze_transactions.select(
    "account_id",
    "event",
    "time_since_test_start",
    col("value.amount").alias("amount"),
    coalesce(
        col("value.`offer id`"),
        col("value.offer_id")
    ).alias("offer_id"), # Transformação: Junta os campos em uma única coluna
    col("value.reward").alias("reward")
)

# Visualização da nova estrutura do DataFrame
silver_transactions.show(truncate=False)

+--------------------------------+--------------+---------------------+------+--------------------------------+------+
|account_id                      |event         |time_since_test_start|amount|offer_id                        |reward|
+--------------------------------+--------------+---------------------+------+--------------------------------+------+
|78afa995795e4d85b5d9ceeca43f5fef|offer received|0.0                  |NULL  |9b98b8c7a33c4b65b9aebfe6a799e6d9|NULL  |
|a03223e636434f42ac4c3df47e8bac43|offer received|0.0                  |NULL  |0b1e1539f2cc45b7b9fa7c272da2e1d7|NULL  |
|e2127556f4f64592b11af22de27a7932|offer received|0.0                  |NULL  |2906b810c7d4411798c6938adc9daaa5|NULL  |
|8ec6ce2a7e7949b1bf142def7d0e0586|offer received|0.0                  |NULL  |fafdcd668e3743c1bb461111dcafc2a4|NULL  |
|68617ca6246f4fbc85e91a2a49552598|offer received|0.0                  |NULL  |4d5c57ea9a6940dd891ad53e9dbe8da0|NULL  |
|389bc3fa690240e798340f5a15918d5c|offer received

In [0]:
# Validação: Tipos de dados das colunas do novo DataFrame
silver_transactions

DataFrame[account_id: string, event: string, time_since_test_start: double, amount: double, offer_id: string, reward: double]

In [0]:
# Transformação: Converter tipos de dados
from pyspark.sql.functions import to_date, col

silver_transactions = (
    silver_transactions
    .withColumn(
        "reward",
        col("reward").cast("decimal(10,2)") # Campo double para decimal
    )
    .withColumn(
        "amount",
        col("amount").cast("decimal(10,2)") # Campo double para int
    )
    .withColumn(
        "time_since_test_start",
        col("time_since_test_start").cast("int") # Campo double para int
    )
)

# Visualização dos novos tipos de dados
silver_transactions

DataFrame[account_id: string, event: string, time_since_test_start: int, amount: decimal(10,2), offer_id: string, reward: decimal(10,2)]

In [0]:
gold_transactions = silver_transactions.dropDuplicates()
# Visualização da tabela final
gold_transactions.count()
# 428 registros duplicados

306106

In [0]:
# Transformação: Criação da coluna ID único das transações
from pyspark.sql.functions import expr

gold_transactions = gold_transactions.withColumn("transaction_id", expr("uuid()"))

#Visualização do novo DataFrame
gold_transactions.show()

+--------------------+--------------+---------------------+------+--------------------+------+--------------------+
|          account_id|         event|time_since_test_start|amount|            offer_id|reward|      transaction_id|
+--------------------+--------------+---------------------+------+--------------------+------+--------------------+
|66e04cebe10343a9b...|offer received|                    0|  NULL|5a8bc65990b245e5a...|  NULL|2157c1aa-c57e-409...|
|fad67e30e00241c9b...|offer received|                    0|  NULL|9b98b8c7a33c4b65b...|  NULL|72a999fd-ca63-4c9...|
|9538885fe02043eca...|offer received|                    0|  NULL|4d5c57ea9a6940dd8...|  NULL|5573051f-16e5-4df...|
|0b6b453772ea4c3a9...|offer received|                    0|  NULL|5a8bc65990b245e5a...|  NULL|5a705cfb-70cd-4b6...|
|a885874d0aae40fca...|offer received|                    0|  NULL|fafdcd668e3743c1b...|  NULL|456da3c2-24e8-474...|
|5d02eeee2ca447b38...|offer received|                    0|  NULL|3f207d

In [0]:
# Checagem: Quais são os possíveis preenchimentos do campo Evento? 
display(
    gold_transactions.groupBy("event")
      .count()
      .orderBy(col("count").desc())
)
#Campo já normalizado

event,count
transaction,138930
offer received,76277
offer viewed,57725
offer completed,33174


In [0]:
# Análise de um cliente
cliente_1 = gold_transactions.filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f").orderBy(col("time_since_test_start"))
cliente_1.show(n=50,truncate=False)

+--------------------------------+---------------+---------------------+------+--------------------------------+------+------------------------------------+
|account_id                      |event          |time_since_test_start|amount|offer_id                        |reward|transaction_id                      |
+--------------------------------+---------------+---------------------+------+--------------------------------+------+------------------------------------+
|9fa9ae8f57894cc9a3b8a9bbe0fc1b2f|transaction    |0                    |34.56 |NULL                            |NULL  |0dd03ffb-9682-4e6d-9233-bc42db55997f|
|9fa9ae8f57894cc9a3b8a9bbe0fc1b2f|offer viewed   |0                    |NULL  |2906b810c7d4411798c6938adc9daaa5|NULL  |e41d7d58-c17e-4f7d-8a17-fde6e1234ad3|
|9fa9ae8f57894cc9a3b8a9bbe0fc1b2f|offer received |0                    |NULL  |2906b810c7d4411798c6938adc9daaa5|NULL  |c9d77418-c5e7-4486-befd-21c4e169b8d2|
|9fa9ae8f57894cc9a3b8a9bbe0fc1b2f|offer completed|0       

In [0]:
# Visualização da tabela final
gold_transactions.show()

+--------------------+--------------+---------------------+------+--------------------+------+--------------------+
|          account_id|         event|time_since_test_start|amount|            offer_id|reward|      transaction_id|
+--------------------+--------------+---------------------+------+--------------------+------+--------------------+
|66e04cebe10343a9b...|offer received|                    0|  NULL|5a8bc65990b245e5a...|  NULL|a1f665a9-98f9-43b...|
|fad67e30e00241c9b...|offer received|                    0|  NULL|9b98b8c7a33c4b65b...|  NULL|f94cf473-5cff-4cf...|
|9538885fe02043eca...|offer received|                    0|  NULL|4d5c57ea9a6940dd8...|  NULL|27d64747-1e12-479...|
|0b6b453772ea4c3a9...|offer received|                    0|  NULL|5a8bc65990b245e5a...|  NULL|7f3afa56-b485-426...|
|a885874d0aae40fca...|offer received|                    0|  NULL|fafdcd668e3743c1b...|  NULL|027c1edf-8665-462...|
|5d02eeee2ca447b38...|offer received|                    0|  NULL|3f207d

## Etapa 3: Construção do dataset unificado

**Objetivo**: Unir as 3 tabelas construídas acima em uma base única para analisar o histórico de transações, ofertas e clientes

In [0]:
# Transformação: Unir as tabelas finais de transações e perfil do cliente
transactions_profile = gold_transactions.join(gold_profile, gold_transactions.account_id == gold_profile.id , how="inner")

# Escolha do inner: Todas as transações têm que ter um perfil do cliente associado
    # Com essa escolha, as transações de clientes com 118 anos foram excluídas da tabela

In [0]:
# Transformação: Unir a tabela final de ofertas e a tabela unificada de transações e perfil
transactions_profile_offers = transactions_profile.join(gold_offers, transactions_profile.offer_id == gold_offers.id, "left")

# Escolha do left: Nem toda transação tem uma oferta associada 

In [0]:
# Transformação: Seleção das colunas relevantes das tabelas unidas
gold_offer_engagement = transactions_profile_offers.select ("transaction_id", #ID da transação
                                              "event", # Evento da transação
                                              "amount", #Valor da transação
                                              "reward", # Desconto
                                              "time_since_test_start", #Tempo desde o início da oferta
                                              "account_id", #ID referente ao cliente
                                              "age", # Idade
                                              "gender", # Gênero
                                              "credit_card_limit", #Limite do cartão de crédito
                                              "registered_on", # Data da criação da conta
                                              "offer_id", # ID da oferta
                                              "offer_type", # Tipo da oferta
                                              "channels", # Canais da oferta
                                              "discount_value", # Valor do desconto 
                                              "duration", # Duração da oferta
                                              "min_value" # Valor mínimo da compra
                                              )

In [0]:
# Visualização da tabela unificada das tabelas transações, perfil e ofertas
gold_offer_engagement.show()

+--------------------+--------------+------+------+---------------------+--------------------+---+------+-----------------+-------------+--------------------+-------------+--------------------+--------------+--------+---------+
|      transaction_id|         event|amount|reward|time_since_test_start|          account_id|age|gender|credit_card_limit|registered_on|            offer_id|   offer_type|            channels|discount_value|duration|min_value|
+--------------------+--------------+------+------+---------------------+--------------------+---+------+-----------------+-------------+--------------------+-------------+--------------------+--------------+--------+---------+
|f94cf473-5cff-4cf...|offer received|  NULL|  NULL|                    0|fad67e30e00241c9b...| 50|     M|         35000.00|   2014-09-17|9b98b8c7a33c4b65b...|         bogo|[web, email, mobile]|          5.00|       7|     5.00|
|27d64747-1e12-479...|offer received|  NULL|  NULL|                    0|9538885fe02043e

In [0]:
# Transformação: O evento transaction não tem oferta associada, por isso é importante realizar uma transformação para associar as transações que tiveram uma oferta associada
    # Como identificar se a transação tem uma oferta associada?
        # Regras de negócio: 
            # Cliente recebeu uma oferta  
            # Cliente realizou uma transação que atende aos critérios:
                # Valor (amount) maior ou igual ao valor mínimo da oferta (min_value);
                # A diferença entre tempo do teste (time_since_test_start) da transação e do recebimento da oferta tem que ser menor ou igual a duração da oferta
            # Cliente teve um evento de oferta completada/utilizada no mesmo dia da transação (time_since_test_start)
            # O cliente pode utilizar mais de uma oferta na mesma transação

# Exemplo: DataFrame -> transações de um cliente
gold_offer_engagement.filter(
    col("account_id") == '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f'
).select( # Selecionando apenas as colunas que quero visualizar
    "transaction_id",
    "event",
    "amount",
    "reward",
    "time_since_test_start",
    "offer_id",
    "discount_value",
    "duration",
    "min_value"
).orderBy(
    col("time_since_test_start"),
    col("event") # Ordenação para que o evento transaction seja o último por dia desde o início do teste
).show(n=50,truncate=False)

+------------------------------------+---------------+------+------+---------------------+--------------------------------+--------------+--------+---------+
|transaction_id                      |event          |amount|reward|time_since_test_start|offer_id                        |discount_value|duration|min_value|
+------------------------------------+---------------+------+------+---------------------+--------------------------------+--------------+--------+---------+
|fc8cb11e-0888-467e-9cac-8eaae689a894|offer completed|NULL  |2.00  |0                    |2906b810c7d4411798c6938adc9daaa5|2.00          |7       |10.00    |
|c9d77418-c5e7-4486-befd-21c4e169b8d2|offer received |NULL  |NULL  |0                    |2906b810c7d4411798c6938adc9daaa5|2.00          |7       |10.00    |
|e41d7d58-c17e-4f7d-8a17-fde6e1234ad3|offer viewed   |NULL  |NULL  |0                    |2906b810c7d4411798c6938adc9daaa5|2.00          |7       |10.00    |
|0dd03ffb-9682-4e6d-9233-bc42db55997f|transaction   

In [0]:
from pyspark.sql.functions import col

# DataFrame referente aos eventos de ofertas recebidas (offer received)
offers_received = gold_offer_engagement.filter(col("event") == "offer received") \
    .select(
        "account_id",
        "offer_id",
        "time_since_test_start",
        "min_value",
        "duration",
        "offer_type",
        "channels",
        "discount_value"
    ).withColumnRenamed("time_since_test_start", "time_offer_received")

# DataFrame referente aos eventos de transação (transaction)
transactions = gold_offer_engagement.filter(col("event") == "transaction") \
    .select(
        "account_id",
        "transaction_id",
        "amount",
        "time_since_test_start"
    )

# DataFrame referente aos eventos de oferta completada (offer completed)
offers_completed = gold_offer_engagement.filter(col("event") == "offer completed") \
    .select(
        "account_id",
        "offer_id",
        "time_since_test_start"
    ).withColumnRenamed("time_since_test_start", "time_offer_completed")

# União dos eventos de transações com as ofertas recebidas pelo mesmo cliente
transactions_with_offers = transactions.join(
    offers_received,
    on="account_id"
).filter(
    (col("amount") >= col("min_value")) &
    ((col("time_since_test_start") - col("time_offer_received")) <= col("duration"))
)

# Filtro das transações que têm uma oferta completada no mesmo dia
transactions_with_offers_final = transactions_with_offers.join(
    offers_completed,
    on=["account_id", "offer_id"]
).filter(
    col("time_since_test_start") == col("time_offer_completed")
)

# Dataframe final
transactions_with_offers_final = transactions_with_offers_final.select(
    "account_id",
    "transaction_id",
    "offer_id",
    "amount",
    "time_since_test_start",
    "min_value",
    "duration",
    "offer_type",
    "channels",
    "discount_value"
).distinct()

# Visualização da tabela de transações associadas às ofertas utilizadas
transactions_with_offers_final.show(truncate=False)

+--------------------------------+------------------------------------+--------------------------------+------+---------------------+---------+--------+----------+----------------------------+--------------+
|account_id                      |transaction_id                      |offer_id                        |amount|time_since_test_start|min_value|duration|offer_type|channels                    |discount_value|
+--------------------------------+------------------------------------+--------------------------------+------+---------------------+---------+--------+----------+----------------------------+--------------+
|0056df74b63b4298809f0b375a304cf4|2ce01cc5-248f-402f-bf41-ae1a472f0d19|0b1e1539f2cc45b7b9fa7c272da2e1d7|27.59 |17                   |20.00    |10      |discount  |[web, email]                |5.00          |
|c03e92c14ebd48f9b3d247ed6b732069|8279079e-4221-4ab5-b888-58cc236d71e7|0b1e1539f2cc45b7b9fa7c272da2e1d7|31.20 |26                   |20.00    |10      |discount  |[web,

In [0]:
# Visualização das transações com ofertas do cliente selecionado
transactions_with_offers_final.filter(col("account_id") == 'e7fd200255cf48e1a75c4316da554941').show(truncate=False)

+--------------------------------+------------------------------------+--------------------------------+------+---------------------+---------+--------+----------+----------------------------+--------------+
|account_id                      |transaction_id                      |offer_id                        |amount|time_since_test_start|min_value|duration|offer_type|channels                    |discount_value|
+--------------------------------+------------------------------------+--------------------------------+------+---------------------+---------+--------+----------+----------------------------+--------------+
|e7fd200255cf48e1a75c4316da554941|f2000f48-5272-45c3-ac37-54ea77fb9f07|2298d6c36e964ae4a3e7e9706d1fb8c2|26.61 |28                   |7.00     |7       |discount  |[web, email, mobile, social]|3.00          |
|e7fd200255cf48e1a75c4316da554941|b4ad42e8-fc41-4024-95e5-8590a4371e4f|2906b810c7d4411798c6938adc9daaa5|32.81 |23                   |10.00    |7       |discount  |[web,

In [0]:
# Criar um Dataframe único
customer_engagement = (
    gold_offer_engagement.alias("oe")
    .join(
        transactions_with_offers_final.alias("t"),
        on = "transaction_id",
        how = "left"
    )
    .select(
        "oe.transaction_id",
        "oe.event",
        "oe.amount",
        "oe.reward",
        "oe.time_since_test_start",
        "oe.account_id",
        "oe.age",
        "oe.gender",
        "oe.credit_card_limit",
        "oe.registered_on",
        coalesce("t.offer_id", "oe.offer_id").alias("offer_id"),
        coalesce("t.offer_type","oe.offer_type").alias("offer_type"),
        coalesce("t.channels","oe.channels").alias("channels"),
        coalesce("t.discount_value","oe.discount_value").alias("discount_value"),
        coalesce("t.duration","oe.duration").alias("duration"),
        coalesce("t.min_value","oe.min_value").alias("min_value")
    )
)

# Visualização da tabela final
customer_engagement.show()

+--------------------+---------------+------+------+---------------------+--------------------+---+------+-----------------+-------------+--------------------+-------------+--------------------+--------------+--------+---------+
|      transaction_id|          event|amount|reward|time_since_test_start|          account_id|age|gender|credit_card_limit|registered_on|            offer_id|   offer_type|            channels|discount_value|duration|min_value|
+--------------------+---------------+------+------+---------------------+--------------------+---+------+-----------------+-------------+--------------------+-------------+--------------------+--------------+--------+---------+
|f94cf473-5cff-4cf...| offer received|  NULL|  NULL|                    0|fad67e30e00241c9b...| 50|     M|         35000.00|   2014-09-17|9b98b8c7a33c4b65b...|         bogo|[web, email, mobile]|          5.00|       7|     5.00|
|027c1edf-8665-462...| offer received|  NULL|  NULL|                    0|5120299298

In [0]:
# Salvar o arquivo final
dataset_unificado = customer_engagement  

caminho_arquivo = "/Volumes/workspace/raw/ifood_case/data/processed/"

dataset_unificado.write \
    .mode("overwrite") \
    .json(caminho_arquivo)

files = dbutils.fs.ls(caminho_arquivo)

# Buscar o nome do arquivo JSON que inicia com "part-*"
dataset_unificado_json = [
    f.path for f in files 
    if f.name.startswith("part-") and f.name.endswith(".json")
][0]

print("Arquivo JSON salvo:")
print(dataset_unificado_json)

# Baixar arquivo via UI
displayHTML(f"""
<p>Caminho para baixar o arquivo JSON final:</p>
<li><b>Data → Catalog → Volumes → workspace.raw.ifood_case → data → processed </b></li>
<li>Nome do arquivo: <b>{dataset_unificado_json.split('/')[-1]}</b></li>
</ul>
""")


Arquivo JSON salvo:
dbfs:/Volumes/workspace/raw/ifood_case/data/processed/part-00000-tid-117763214128754737-d0a1fcc4-bda3-428d-ac5b-d304cd7a59d6-753-1-c000.json


Caminho para baixar o arquivo JSON final: 
 Data → Catalog → Volumes → workspace.raw.ifood_case → data → processed 
 Nome do arquivo: part-00000-tid-117763214128754737-d0a1fcc4-bda3-428d-ac5b-d304cd7a59d6-753-1-c000.json

In [0]:
#Armazenamento na tabela Delta
customer_engagement.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.gold.customer_engagement") #Tabela final

## Etapa 4: Análise da base final

###  Características da base

In [0]:
%sql
select
  count(distinct transaction_id) as qtde_tl_transacoes,
  count(case when event = 'transaction' then 1 else 0 end) as qtde_tl_compras,
  count(distinct account_id) as qtde_tl_clientes,
  sum(amount) as tl_receita
from
  main.gold.customer_engagement

qtde_tl_transacoes,qtde_tl_compras,qtde_tl_clientes,tl_receita
272363,274670,14825,1798006.79


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
with base_cartao as
(select
  distinct account_id,
  credit_card_limit
from
  main.gold.customer_engagement)

select
  round(avg(credit_card_limit),2) as avg_credit_card_limit
from
  base_cartao


avg_credit_card_limit
65404.99


In [0]:
%sql
with usuarios as
(select
  distinct account_id,
  age
from
  main.gold.customer_engagement)

select
  round(avg(age),2) as avg_idade
from
  usuarios

avg_idade
54.39


In [0]:
%sql
WITH base_compras as
(select
  account_id,
  count(*) as qtde,
  sum(amount) as valor
from
  main.gold.customer_engagement
where
  event = 'transaction'
group by
  1)

SELECT 
  round(avg(qtde),2) as avg_qtde_compras,
  round(avg(valor),2) as avg_valor_compras
from 
  base_compras

avg_qtde_compras,avg_valor_compras
8.71,124.07


In [0]:
%sql
with base_idade as
(select 
  case when 20 > age then '1. Abaixo de 20' 
       when age between 20 and 29 then '2. Faixa 20 a 29'
       when age between 30 and 39 then '3. Faixa 30 a 39'
       when age between 40 and 49 then '4. Faixa 40 a 49'
       when age between 50 and 59 then '5. Faixa 50 a 59'
       when age between 60 and 69 then '6. Faixa 60 a 69'
       when age between 70 and 79 then '7. Faixa 70 a 79'
       when age >= 80 then '8. Acima de 80'
       else '-' end as faixa_idade,
  count(distinct account_id) as qtde_clientes
from 
  main.gold.customer_engagement
group by
  1)

select
  faixa_idade,
  qtde_clientes,
  round(qtde_clientes*100/ SUM(qtde_clientes) OVER (), 2) AS prct_clientes
from
  base_idade
order by
  1

faixa_idade,qtde_clientes,prct_clientes
1. Abaixo de 20,205,1.38
2. Faixa 20 a 29,1369,9.23
3. Faixa 30 a 39,1526,10.29
4. Faixa 40 a 49,2309,15.58
5. Faixa 50 a 59,3541,23.89
6. Faixa 60 a 69,2991,20.18
7. Faixa 70 a 79,1782,12.02
8. Acima de 80,1102,7.43


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
with base_genero as
(select 
  gender,
  count(distinct account_id) as qtde_clientes
from 
  main.gold.customer_engagement
group by
  1)

select
  gender as genero,
  qtde_clientes,
  round(qtde_clientes*100/ SUM(qtde_clientes) OVER (), 2) AS prct_clientes
from
  base_genero
order by
  1

genero,qtde_clientes,prct_clientes
F,6129,41.34
M,8484,57.23
O,212,1.43


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select 
  CASE
        WHEN credit_card_limit < 30000 THEN 'Abaixo de 30.000'
        WHEN credit_card_limit BETWEEN 30000 AND 39999.99 THEN '30.000 a 39.999'
        WHEN credit_card_limit BETWEEN 40000 AND 49999.99 THEN '40.000 a 49.999'
        WHEN credit_card_limit BETWEEN 50000 AND 59999.99 THEN '50.000 a 59.999'
        WHEN credit_card_limit BETWEEN 60000 AND 69999.99 THEN '60.000 a 69.999'
        WHEN credit_card_limit BETWEEN 70000 AND 79999.99 THEN '70.000 a 79.999'
        WHEN credit_card_limit BETWEEN 80000 AND 89999.99 THEN '80.000 a 89.999'
        WHEN credit_card_limit BETWEEN 90000 AND 99999.99 THEN '90.000 a 99.999'
        WHEN credit_card_limit BETWEEN 100000 AND 109999.99 THEN '100.000 a 109.999'
        WHEN credit_card_limit BETWEEN 110000 AND 120000 THEN '110.000 a 120.000'
        ELSE 'Acima de 120.000'
    END AS faixa_limite_cartao,
  CASE
            WHEN credit_card_limit < 30000 THEN 1
            WHEN credit_card_limit BETWEEN 30000 AND 39999.99 THEN 2
            WHEN credit_card_limit BETWEEN 40000 AND 49999.99 THEN 3
            WHEN credit_card_limit BETWEEN 50000 AND 59999.99 THEN 4
            WHEN credit_card_limit BETWEEN 60000 AND 69999.99 THEN 5
            WHEN credit_card_limit BETWEEN 70000 AND 79999.99 THEN 6
            WHEN credit_card_limit BETWEEN 80000 AND 89999.99 THEN 7
            WHEN credit_card_limit BETWEEN 90000 AND 99999.99 THEN 8
            WHEN credit_card_limit BETWEEN 100000 AND 109999.99 THEN 9
            WHEN credit_card_limit BETWEEN 110000 AND 120000 THEN 10
            ELSE 11
        END AS ordem,
    count(distinct account_id) as qtde_clientes
from 
  main.gold.customer_engagement
group by
  faixa_limite_cartao,
  ordem
order by
  ordem

faixa_limite_cartao,ordem,qtde_clientes
30.000 a 39.999,2,1940
40.000 a 49.999,3,1841
50.000 a 59.999,4,2661
60.000 a 69.999,5,2328
70.000 a 79.999,6,2338
80.000 a 89.999,7,1409
90.000 a 99.999,8,1215
100.000 a 109.999,9,575
110.000 a 120.000,10,518


Databricks visualization. Run in Databricks to view.

### Análises de ofertas

In [0]:
%sql
-- Todos os clientes da base receberam uma oferta?
with usuarios_com_oferta as 
(select
  distinct account_id
from
  main.gold.customer_engagement
where
  event = 'offer received'),

sem_ofertas as
(select
  distinct e.account_id
from
  main.gold.customer_engagement e 
  left join usuarios_com_oferta u on u.account_id = e.account_id
where
  u.account_id is null)

select
  count(distinct o.account_id) as qtde_clientes
from
  sem_ofertas o 
-- 5 clientes não receberam ofertas

qtde_clientes
5


In [0]:
%sql
-- Qual a oferta mais enviada ao cliente e qual mais converteu?
with offer_received as 
(select
  offer_id,
  offer_type,
  min_value,
  discount_value,
  duration,
  count(distinct transaction_id) as qtde_offer_received
from
  main.gold.customer_engagement
where
  event = 'offer received'
group by
  offer_id,
  offer_type,
  min_value,
  discount_value,
  duration),

transaction_with_offer as 
(select
  offer_id,
  offer_type,
  count(distinct transaction_id) as qtde_compras,
  sum(amount) as total_receita
from
  main.gold.customer_engagement
where
  event = 'transaction'
group by
  offer_id,
  offer_type)

select
  ofr.*,
  tr.qtde_compras,
  (tr.qtde_compras/ofr.qtde_offer_received)::decimal(10,5) as prct_conversao_oferta,
  tr.total_receita,
  qtde_compras*discount_value as total_desconto,
  (total_receita -total_desconto)::Decimal(10,2) as receita_liquida
from
  offer_received ofr
  join transaction_with_offer tr on tr.offer_id = ofr.offer_id
order by
  prct_conversao_oferta desc

--Análise:
  --Embora as ofertas do tipo BOGO apresentem maior volume de transações, as ofertas de desconto geram maior retorno financeiro.
  --As ofertas do tipo BOGO, apesar do maior número de transações, exigem um valor mínimo de compra inferior.
    

offer_id,offer_type,min_value,discount_value,duration,qtde_offer_received,qtde_compras,prct_conversao_oferta,total_receita,total_desconto,receita_liquida
f19421c1d4aa40978ebb69ca19b0e20d,bogo,5.00,5.00,5,6576,4702,0.71502,93103.48,23510.00,69593.48
2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7.00,3.00,7,6655,4574,0.68730,95177.13,13722.00,81455.13
9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5.00,5.00,7,6685,4593,0.68706,84556.73,22965.00,61591.73
fafdcd668e3743c1bb461111dcafc2a4,discount,10.00,2.00,10,6652,4247,0.63845,101019.97,8494.00,92525.97
ae264e3637204a6fb9bb56bc8210ddfd,bogo,10.00,10.00,7,6683,4065,0.60826,95346.97,40650.00,54696.97
4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10.00,10.00,5,6593,3681,0.55832,86502.16,36810.00,49692.16
2906b810c7d4411798c6938adc9daaa5,discount,10.00,2.00,7,6631,3626,0.54683,86790.73,7252.00,79538.73
0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20.00,5.00,10,6726,2051,0.30494,73594.19,10255.00,63339.19


In [0]:
%sql
-- Qual o perfil das pessoas que utilizaram a oferta que trouxe mais retorno financeiro? offer_id: fafdcd668e3743c1bb461111dcafc2a4
-- E qual a média de idade e limite de cartão de crédito para oferta com maior conversão? offer_id: f19421c1d4aa40978ebb69ca19b0e2
with clientes as
(select
  distinct 
  offer_id,
  account_id,
  age,
  credit_card_limit,
  gender
from
  main.gold.customer_engagement
where
  offer_id in ('fafdcd668e3743c1bb461111dcafc2a4','f19421c1d4aa40978ebb69ca19b0e20d')
  and event = 'transaction')

select
  offer_id,
  round(avg(age),2) as avg_idade,
  round(avg(credit_card_limit),2) as avg_credit_card_limit
from
  clientes
group by
  offer_id

offer_id,avg_idade,avg_credit_card_limit
fafdcd668e3743c1bb461111dcafc2a4,57.71,73103.62
f19421c1d4aa40978ebb69ca19b0e20d,55.49,68349.52


In [0]:
%sql
-- O gênero dos clientes que utilizaram a oferta que trouxe mais retorno financeiro é relevante? offer_id: fafdcd668e3743c1bb461111dcafc2a4
-- E o gênero dos clientes que utilizaram a oferta com maior conversão? offer_id: f19421c1d4aa40978ebb69ca19b0e2
with clientes as
(select
  distinct 
  offer_id,
  account_id,
  age,
  credit_card_limit,
  gender
from
  main.gold.customer_engagement
where
  offer_id in ('fafdcd668e3743c1bb461111dcafc2a4','f19421c1d4aa40978ebb69ca19b0e20d')
  and event = 'transaction')

select
  offer_id,
  gender,
  count(distinct account_id) as qtde_clientes
from
  clientes
group by
  offer_id,
  gender
order by
  offer_id,
  qtde_clientes

-- Para a oferta f19421c1d4aa40978ebb69ca19b0e20d, o gênero dos clientes acompanha a distribuição geral
-- Para a oferta fafdcd668e3743c1bb461111dcafc2a4, o gênero dos clientes é diferente da distribuição geral. Tem mais mulheres na base, cerca de 50%, enquanto na base geral é ~41%

offer_id,gender,qtde_clientes
f19421c1d4aa40978ebb69ca19b0e20d,O,59
f19421c1d4aa40978ebb69ca19b0e20d,F,1642
f19421c1d4aa40978ebb69ca19b0e20d,M,1861
fafdcd668e3743c1bb461111dcafc2a4,O,48
fafdcd668e3743c1bb461111dcafc2a4,M,1556
fafdcd668e3743c1bb461111dcafc2a4,F,1629


## Etapa 5: Baixar requirements

In [0]:
%pip freeze > /Workspace/ifood-case/requirements.txt